In [53]:
import pandas as pd
import numpy as np

In [193]:
old_df = pd.read_csv('../data/pedestrian-counting-system-monthly-2009-to-2022.csv')
new_df = pd.read_csv('../data/pedestrian-counting-system-monthly-2023.csv')
sensor_locs = pd.read_csv('../data/pedestrian-counting-system-sensor-locations.csv')

In [194]:
new_df['SensingDateTime(Hour)'] = new_df['SensingDateTime(Hour)'].apply(lambda x: x[:-6])

In [195]:
old_df.head()

,ID,Date_Time,Year,Month,Mdate,Day,Time,Sensor_ID,Sensor_Name,Hourly_Counts
0,2887628,"November 01, 2019 05:00:00 PM",2019,November,1,Friday,17,34,Flinders St-Spark La,300
1,2887629,"November 01, 2019 05:00:00 PM",2019,November,1,Friday,17,39,Alfred Place,604
2,2887630,"November 01, 2019 05:00:00 PM",2019,November,1,Friday,17,37,Lygon St (East),216
3,2887631,"November 01, 2019 05:00:00 PM",2019,November,1,Friday,17,40,Lonsdale St-Spring St (West),627
4,2887632,"November 01, 2019 05:00:00 PM",2019,November,1,Friday,17,36,Queen St (West),774


In [196]:
new_df.head()

,SensingDateTime(Hour),LocationID,Direction_1,Direction_2,Total_of_Directions
0,2023-09-01T03:00:00,1,5,14,19
1,2023-09-01T05:00:00,1,9,6,15
2,2023-09-01T09:00:00,1,322,384,706
3,2023-09-01T10:00:00,1,597,515,1112
4,2023-09-01T11:00:00,1,813,652,1465


In [197]:
old_df['Parsed_Time'] = pd.to_datetime(old_df['Date_Time'], format='%B %d, %Y %I:%M:%S %p', errors='raise')
new_df['Parsed_Time'] = pd.to_datetime(new_df['SensingDateTime(Hour)'], format='%Y-%m-%dT%H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S')

old_df['Hour'] = old_df['Parsed_Time'].dt.hour
new_df['Hour'] = pd.to_datetime(new_df['SensingDateTime(Hour)'], format='%Y-%m-%dT%H:%M:%S').dt.hour

In [198]:
clean_old = old_df.drop(columns=['ID', 'Year', 'Month','Mdate','Day','Time', 'Sensor_Name','Date_Time'])
clean_new = new_df.drop(columns=['Direction_1','Direction_2','SensingDateTime(Hour)'])
clean_new.rename({"Total_of_Directions": "Hourly_Counts", "LocationID": "Sensor_ID"}, axis=1, inplace=True)

In [199]:
clean_old.head()

,Sensor_ID,Hourly_Counts,Parsed_Time,Hour
0,34,300,2019-11-01 17:00:00,17
1,39,604,2019-11-01 17:00:00,17
2,37,216,2019-11-01 17:00:00,17
3,40,627,2019-11-01 17:00:00,17
4,36,774,2019-11-01 17:00:00,17


In [200]:
clean_new.head()

,Sensor_ID,Hourly_Counts,Parsed_Time,Hour
0,1,19,2023-09-01 03:00:00,3
1,1,15,2023-09-01 05:00:00,5
2,1,706,2023-09-01 09:00:00,9
3,1,1112,2023-09-01 10:00:00,10
4,1,1465,2023-09-01 11:00:00,11


In [201]:
comb = pd.concat([clean_old, clean_new])

In [202]:
comb.sort_values(['Sensor_ID', 'Hour']).head(15)

,Sensor_ID,Hourly_Counts,Parsed_Time,Hour
363,1,205,2019-11-02 00:00:00,0
1687,1,247,2019-11-03 00:00:00,0
2961,1,55,2019-11-04 00:00:00,0
4237,1,129,2019-11-05 00:00:00,0
5534,1,79,2019-11-06 00:00:00,0
6830,1,44,2019-11-07 00:00:00,0
8126,1,48,2019-11-08 00:00:00,0
9427,1,155,2019-11-09 00:00:00,0
10727,1,242,2019-11-10 00:00:00,0
12026,1,70,2019-11-11 00:00:00,0


In [203]:
output = pd.DataFrame(columns=['Sensor_ID', 'Hour', 'Count'])
sensor_ids = comb['Sensor_ID'].unique()
hours = comb['Hour'].unique()
for id in sensor_ids:
    for hour in hours:
        ftr = comb[(comb['Sensor_ID'] == id) & (comb['Hour'] == hour)]
        count = ftr['Hourly_Counts'].sum()
        output.loc[len(output.index)] = [id, hour, count]

In [204]:
output['Average'] = output['Count'].apply(lambda x: x/14)

In [205]:
output_new = output[output['Sensor_ID'].isin(sensor_locs['Location_ID'].unique())]

In [206]:
output_new.head()

,Sensor_ID,Hour,Count,Average
24,39,17,902474,64462.428571
25,39,18,620956,44354.000000
26,39,19,392025,28001.785714
27,39,16,682070,48719.285714
28,39,20,266163,19011.642857


In [207]:
def find_lat(x):
    y = sensor_locs[sensor_locs['Location_ID'] == x]
    return y['Latitude'].item()
def find_long(x):
    y = sensor_locs[sensor_locs['Location_ID'] == x]
    return y['Longitude'].item()
def find_name(x):
    y = sensor_locs[sensor_locs['Location_ID'] == x]
    return y['Sensor_Description'].item()

In [208]:
output_new['Latitude']= output_new['Sensor_ID'].apply(find_lat)
output_new['Longitude']= output_new['Sensor_ID'].apply(find_long)
output_new['Name'] = output_new['Sensor_ID'].apply(find_name)

C:\Users\Ethan\AppData\Local\Temp\ipykernel_38080\3563477010.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_new['Latitude']= output_new['Sensor_ID'].apply(find_lat)
C:\Users\Ethan\AppData\Local\Temp\ipykernel_38080\3563477010.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_new['Longitude']= output_new['Sensor_ID'].apply(find_long)
C:\Users\Ethan\AppData\Local\Temp\ipykernel_38080\3563477010.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [209]:
output_new.to_csv('../data/pedestrian_counting.csv')